In [261]:
import pandas as pd
import numpy as np
import pickle
import os
import copy


### VISUALIZANDO O DATASET FORNECIDO

In [262]:
df = pd.read_csv('Ansatz_training_reports(CostaSH).csv')

In [263]:
df.head()

,ARQUITETURA_ANSATZ,INPUT_EMBEDDING,DATASET,DATASET_DIVISION_INDEX,OPTIMIZER,UNSUPERVISED_METRIC,MEASURED_WIRES,MEASURE_TYPE,TRAIN_METRIC_COST,TEST_METRIC_COST,TRAIN_ACCURACY,TEST_ACCURACY,WEIGHT,BIAS,USE_BIAS
0,hr,amplitude,blobs_2classes_2features_500samples_low_noise,0,SPSAOptimizer(10),silhouette,1,expval,0.687974,0.707721,0.388892,0.357417,[ 0.46310958 1.51730179 -0.8685384 ],[0.26964764],YES
1,hr,amplitude,blobs_2classes_2features_500samples_low_noise,0,SPSAOptimizer(10),calinski harabasz,1,expval,10.000000,10.000000,0.000000,0.000000,[-7.88769834 -6.84109828 7.89266777],[-8.1856726],YES
2,hr,amplitude,blobs_2classes_2features_500samples_low_noise,1,SPSAOptimizer(10),silhouette,1,expval,0.685815,0.725449,0.409554,0.357371,[ 0.49662902 1.46714475 -0.88130919],[0.29223773],YES
3,hr,amplitude,blobs_2classes_2features_500samples_low_noise,1,SPSAOptimizer(10),calinski harabasz,1,expval,10.000000,10.000000,0.000000,0.000000,[ 78.22535209 -76.49117364 -78.70308033],[77.84363186],YES
4,hr,amplitude,blobs_2classes_2features_500samples_low_noise,2,SPSAOptimizer(10),silhouette,1,expval,0.752375,0.691396,0.293520,0.392557,[ 0.49948276 1.45020381 -0.85730261],[0.21516058],YES


In [264]:
# Valores únicos de várias colunas selecionadas
colunas_interesse = ['ARQUITETURA_ANSATZ', 'INPUT_EMBEDDING', 'DATASET_DIVISION_INDEX', 'DATASET', 'OPTIMIZER', 'UNSUPERVISED_METRIC', 'MEASURED_WIRES', 'MEASURE_TYPE', 'USE_BIAS']
valores_unicos = {col: df[col].unique() for col in colunas_interesse}

# Exibindo os resultados
for col, valores in valores_unicos.items():
    print(f"\nValores únicos em {col}:")
    print(valores)
    print(f"Número de valores únicos: {len(valores)}")


Valores únicos em ARQUITETURA_ANSATZ:
['hr' 'ru' 'ruu' 'rx' 'ry' 'rz' 'alpha' 'beta' 'gamma' 'phi' 'theta'
 '0_zero' '1_mochi' '2_yadi' '3_tatu' '4_wana' '5_tano' '6_samanu'
 '7_sambwadi' '8_nake' '9_divwe' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J'
 'K' 'L' 'M' 'N' 'O']
Número de valores únicos: 36

Valores únicos em INPUT_EMBEDDING:
['amplitude' 'phasex']
Número de valores únicos: 2

Valores únicos em DATASET_DIVISION_INDEX:
[0 1 2 3 4]
Número de valores únicos: 5

Valores únicos em DATASET:
['blobs_2classes_2features_500samples_low_noise'
 'blobs_2classes_2features_500samples_no_noise'
 'blobs_2classes_2features_50samples_low_noise'
 'blobs_2classes_2features_50samples_no_noise'
 'circles_500samples_low_noise' 'circles_500samples_no_noise'
 'circles_50samples_low_noise' 'circles_50samples_no_noise'
 'moons_500samples_low_noise' 'moons_500samples_no_noise'
 'moons_50samples_low_noise' 'moons_50samples_no_noise'
 'xor_500samples_low_noise' 'xor_500samples_no_noise'
 'xor_50samples_low_n

In [265]:
# ordenando o DataFrame por 'ARQUITETURA_ANSATZ'
df_sorted = df.sort_values(by=['ARQUITETURA_ANSATZ'])

In [266]:
# verificando a ordem dos valores únicos de 'ARQUITETURA_ANSATZ' após a ordenação
valores_unicos_sorted = df_sorted['ARQUITETURA_ANSATZ'].unique()
print("\nValores únicos de 'ARQUITETURA_ANSATZ' após ordenação:")
print(valores_unicos_sorted)
print(f"Número de valores únicos após ordenação: {len(valores_unicos_sorted)}")


Valores únicos de 'ARQUITETURA_ANSATZ' após ordenação:
['0_zero' '1_mochi' '2_yadi' '3_tatu' '4_wana' '5_tano' '6_samanu'
 '7_sambwadi' '8_nake' '9_divwe' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J'
 'K' 'L' 'M' 'N' 'O' 'alpha' 'beta' 'gamma' 'hr' 'phi' 'ru' 'ruu' 'rx'
 'ry' 'rz' 'theta']
Número de valores únicos após ordenação: 36


### TRANSFORMANDO O DATASET PARA O INPUT DOS MODELOS

In [267]:
def transformar_dataframe(df, file_path, file_metrics = "./../datasets/metrics/",  target="ACURACIA_TESTE"):

    """
    Função para transformar os metadados do treinamento do ansatz nos dados para os modelos clássicos.
    Parâmetros: df: DataFrame com os metadados do treinamento do ansatz
              file_metrics: caminho do arquivo com os metadados do treinamento do ansatz
              file_path: caminho para salvar o arquivo csv
              target: nome da coluna alvo (default: "ACURACIA_TESTE")
    Retorna: Arquivos csv com os dados transformados
    """
    # ordenando o DataFrame por 'ARQUITETURA_ANSATZ'
    df = df.sort_values(by=['ARQUITETURA_ANSATZ'])

    # mapeamento dos valores únicos de 'ARQUITETURA_ANSATZ' para números
    ansatz_mapping = {val: i for i, val in enumerate(df['ARQUITETURA_ANSATZ'].unique())}

    # mapeamento do valores únicos de 'INPUT_EMBEDDING' para números
    embedding_mapping = {val: i for i, val in enumerate(df['INPUT_EMBEDDING'].unique())}

    # mapeamento do valores únicos de 'UNSUPERVISED_METRIC' para números
    unsupervised_mapping = {val: i for i, val in enumerate(df['UNSUPERVISED_METRIC'].unique())}

    # exibindo os mapeamentos
    print("\nMapeamento de 'ARQUITETURA_ANSATZ' para números:")
    for val, i in ansatz_mapping.items():
        print(f"{val}: {i}")
    print("\nMapeamento de 'INPUT_EMBEDDING' para números:")
    for val, i in embedding_mapping.items():
        print(f"{val}: {i}")
    print("\nMapeamento de 'UNSUPERVISED_METRIC' para números:")
    for val, i in unsupervised_mapping.items():
        print(f"{val}: {i}")

    # nomes dos datasets
    datasets = df['DATASET'].unique()

    # Criando uma lista para armazenar as métricas  
    metrics = []

    for dataset in datasets:
        # Carregando as métricas
        with open(file_metrics + dataset + ".pkl", 'rb') as file:
            aux_metrics = pickle.load(file)
        # Adicionando as métricas à lista
        metrics.append(aux_metrics)

    new_df = {
        'f1': [],
        'f1v': [], 
        'f2': [], 
        'f3': [], 
        'f4': [], 
        'l1': [], 
        'l2': [], 
        'l3': [], 
        'n1': [], 
        'n2': [], 
        'n3': [], 
        'n4': [], 
        't1': [], 
        'lsc': [], 
        'density': [], 
        'clsCoef': [], 
        'hubs': [], 
        't2': [], 
        't3': [], 
        't4': [], 
        'c1': [], 
        'c2': [],
        'target': []
    }


    for sample_id in range(5):

        #filtrar pelo número da sample
        df_sample = df[df["DATASET_DIVISION_INDEX"]==sample_id]

        df_best_accuracy = copy.deepcopy(new_df)
        df_best_ansatz = copy.deepcopy(new_df)
        df_unsupervised_metric = copy.deepcopy(new_df)
        df_best_embedding = copy.deepcopy(new_df)

        for metric in metrics:

            #filtrar pelo nome do arquivo
            df_arq_sample = df_sample[df_sample["DATASET"]==metric["arquivo"]]

            #filtrar pelo Bias
            df_arq_sample_bias = df_arq_sample[df_arq_sample["USE_BIAS"]=="YES"]

            #preencher os dataframes
            for key in new_df.keys():

                if key != 'target':

                    df_best_accuracy[key].append(metric[key][sample_id])
                    df_best_ansatz[key].append(metric[key][sample_id])
                    df_unsupervised_metric[key].append(metric[key][sample_id])
                    df_best_embedding[key].append(metric[key][sample_id])

                else:
                    #preencher a coluna target

                    idx = df_arq_sample_bias[target].idxmax()

                    # para a melhor acurácia
                    df_best_accuracy[key].append(df_arq_sample_bias.loc[idx, target])

                    # para o melhor ansatz
                    df_best_ansatz[key].append(ansatz_mapping[df_arq_sample_bias.loc[idx, 'ARQUITETURA_ANSATZ']])

                    # para a melhor métrica não supervisionada
                    df_unsupervised_metric[key].append(unsupervised_mapping[df_arq_sample_bias.loc[idx, 'UNSUPERVISED_METRIC']])

                    # para o melhor embedding
                    df_best_embedding[key].append(embedding_mapping[df_arq_sample_bias.loc[idx, 'INPUT_EMBEDDING']])
        
        # criar o caminho para salvar os arquivos
        file_path_ = f"{file_path}/input_data/experiment_{sample_id}/"
        if not os.path.exists(file_path_):
            os.makedirs(file_path_)

        # salvar os dicionários em arquivos csv
        df_best_accuracy = pd.DataFrame(df_best_accuracy)
        df_best_accuracy.to_csv(file_path_ + "best_accuracy_sample_" + str(sample_id) + ".csv", index=False)

        df_best_ansatz = pd.DataFrame(df_best_ansatz)
        df_best_ansatz.to_csv(file_path_ + "best_ansatz_sample_" + str(sample_id) + ".csv", index=False)

        df_unsupervised_metric = pd.DataFrame(df_unsupervised_metric)
        df_unsupervised_metric.to_csv(file_path_ + "unsupervised_metric_sample_" + str(sample_id) + ".csv", index=False)

        df_best_embedding = pd.DataFrame(df_best_embedding)
        df_best_embedding.to_csv(file_path_ + "best_embedding_sample_" + str(sample_id) + ".csv", index=False)




In [268]:
transformar_dataframe(df=df_sorted, file_path="./", target="TEST_ACCURACY")


Mapeamento de 'ARQUITETURA_ANSATZ' para números:
0_zero: 0
1_mochi: 1
2_yadi: 2
3_tatu: 3
4_wana: 4
5_tano: 5
6_samanu: 6
7_sambwadi: 7
8_nake: 8
9_divwe: 9
A: 10
B: 11
C: 12
D: 13
E: 14
F: 15
G: 16
H: 17
I: 18
J: 19
K: 20
L: 21
M: 22
N: 23
O: 24
alpha: 25
beta: 26
gamma: 27
hr: 28
phi: 29
ru: 30
ruu: 31
rx: 32
ry: 33
rz: 34
theta: 35

Mapeamento de 'INPUT_EMBEDDING' para números:
phasex: 0
amplitude: 1

Mapeamento de 'UNSUPERVISED_METRIC' para números:
silhouette: 0
calinski harabasz: 1
